In [1]:
import geopandas as gpd

In [13]:
import pathlib
import random
from functools import reduce
from collections import defaultdict

import pandas as pd
import geopandas as gpd
import folium
import shapely
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import sklearn.cluster
#import tensorflow as tf
#import xgboost
#from geoband import API

In [9]:
import numpy as np

In [4]:
import folium

In [7]:
import pandas as pd
bike_bucheon = pd.read_csv('./output/bike_bucheon.csv')
bike_bucheon

,Unnamed: 0,자전거대여소명,위도,경도
0,0,공공자전거대여소 부천시청역,37.504938,126.763465
1,1,공공자전거대여소 상동역,37.506069,126.754122
2,2,공공자전거대여소 신중동역,37.503068,126.775751
3,3,공공자전거대여소 부천역,37.484717,126.782677
4,4,공공자전거대여소 송내역,37.488286,126.755075
5,5,공공자전거대여소 부천시청,37.503728,126.766102


In [2]:
bucheon_bus = pd.read_csv('./output/bucheon_bus.csv')
bucheon_bus

,Unnamed: 0,시군명,정류소명,정류소영문명,정류소id,정류소번호,중앙차로여부,관할관청,위치,WGS84위도,WGS84경도
0,8611,부천시,KCC스위첸아파트.한신아파트,"KCC Switzen Apartment, Hanshin Apartment",211000062,12097.0,노변정류장,경기도 부천시,NaN,37.475267,126.799667
1,8612,부천시,KEB하나은행도당동지점,KEB Hana Bank Dodang Branch,212000004,13292.0,노변정류장,경기도 부천시,NaN,37.516817,126.779700
2,8613,부천시,KEB하나은행도당동지점,KEB Hana Bank Dodang Branch,210000373,11256.0,노변정류장,경기도 부천시,NaN,37.516600,126.779817
3,8614,부천시,KT부천지사,KT Bucheon Branch,210000101,11231.0,노변정류장,경기도 부천시,NaN,37.486967,126.776200
4,8615,부천시,KT부천지사,KT Bucheon Branch,210000365,11222.0,노변정류장,경기도 부천시,NaN,37.486867,126.775250
...,...,...,...,...,...,...,...,...,...,...,...
1157,9768,부천시,휴먼시아1단지,Humansia 1-danji,211000261,12495.0,노변정류장,경기도 부천시,NaN,37.477317,126.812783
1158,9769,부천시,휴먼시아1단지후문,Humansia 1-danji Back Gate,212000394,13324.0,노변정류장,경기도 부천시,NaN,37.530917,126.780800
1159,9770,부천시,휴먼시아1단지후문,Humansia Apartment 1-cha Back Gate,212000395,13357.0,노변정류장,경기도 부천시,NaN,37.530717,126.780500
1160,9771,부천시,휴먼시아3단지,Humansia 3-danji,212000378,13224.0,노변정류장,경기도 부천시,NaN,37.533617,126.788300


In [3]:
subway_bucheon = pd.read_csv('./output/subway_bucheon.csv')
subway_bucheon

,Unnamed: 0,역이름,lat,lon
0,0,역곡,37.485140,126.811622
1,1,부천,37.484085,126.780614
2,2,송내,37.487626,126.750856
3,3,소사,37.482755,126.793465
4,4,중동,37.485731,126.763899
5,5,까치울,37.506166,126.804066
6,6,부천종합운동장,37.505452,126.795309
7,7,춘의,37.504187,126.784773
8,8,신중동,37.503064,126.773788
9,9,부천시청,37.504681,126.761429


In [33]:
인구분포도 = gpd.read_file('data/bucheongrid/bucheonjsonjson.geojson')
인구분포도

,gid,val,geometry
0,다사376426,721.0,"MULTIPOLYGON (((126.79423 37.48053, 126.79422 ..."
1,다사372468,111.0,"MULTIPOLYGON (((126.78934 37.51836, 126.78934 ..."
2,다사351459,63.0,"MULTIPOLYGON (((126.76566 37.51010, 126.76565 ..."
3,다사385461,356.0,"MULTIPOLYGON (((126.80411 37.51213, 126.80410 ..."
4,다사352468,200.0,"MULTIPOLYGON (((126.76671 37.51822, 126.76670 ..."
...,...,...,...
5620,다사379488,NaN,"MULTIPOLYGON (((126.79710 37.53643, 126.79709 ..."
5621,다사387492,NaN,"MULTIPOLYGON (((126.80612 37.54009, 126.80611 ..."
5622,다사374464,NaN,"MULTIPOLYGON (((126.79164 37.51476, 126.79163 ..."
5623,다사374455,NaN,"MULTIPOLYGON (((126.79172 37.50665, 126.79171 ..."


In [34]:
인구분포도.drop('geometry', axis=1).describe(include='all')

,gid,val
count,5625,2898.000000
unique,5625,NaN
top,다사369439,NaN
freq,1,NaN
mean,NaN,282.829538
std,NaN,325.746263
min,NaN,0.000000
25%,NaN,56.000000
50%,NaN,235.000000
75%,NaN,400.000000


In [36]:
인구분포도.loc[인구분포도['val'].apply(pd.isna), 'val'] = 0
인구분포도

,gid,val,geometry
0,다사376426,721.0,"MULTIPOLYGON (((126.79423 37.48053, 126.79422 ..."
1,다사372468,111.0,"MULTIPOLYGON (((126.78934 37.51836, 126.78934 ..."
2,다사351459,63.0,"MULTIPOLYGON (((126.76566 37.51010, 126.76565 ..."
3,다사385461,356.0,"MULTIPOLYGON (((126.80411 37.51213, 126.80410 ..."
4,다사352468,200.0,"MULTIPOLYGON (((126.76671 37.51822, 126.76670 ..."
...,...,...,...
5620,다사379488,0.0,"MULTIPOLYGON (((126.79710 37.53643, 126.79709 ..."
5621,다사387492,0.0,"MULTIPOLYGON (((126.80612 37.54009, 126.80611 ..."
5622,다사374464,0.0,"MULTIPOLYGON (((126.79164 37.51476, 126.79163 ..."
5623,다사374455,0.0,"MULTIPOLYGON (((126.79172 37.50665, 126.79171 ..."


In [37]:
인구분포도.drop('geometry', axis=1).describe(include='all')

,gid,val
count,5625,5625.000000
unique,5625,NaN
top,다사369439,NaN
freq,1,NaN
mean,NaN,145.713778
std,NaN,273.207351
min,NaN,0.000000
25%,NaN,0.000000
50%,NaN,0.000000
75%,NaN,244.000000


In [38]:
# Polygon의 중심점에 해당하는 좌표를 파생변수화하여, 위도-경도의 데이터로 뽑아와야 함

인구분포도['경도'] = 인구분포도['geometry'].centroid.x
인구분포도['위도'] = 인구분포도['geometry'].centroid.y
인구분포도

<ipython-input-38-1f38f2caf1b3>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  인구분포도['경도'] = 인구분포도['geometry'].centroid.x
<ipython-input-38-1f38f2caf1b3>:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  인구분포도['위도'] = 인구분포도['geometry'].centroid.y


,gid,val,geometry,경도,위도
0,다사376426,721.0,"MULTIPOLYGON (((126.79423 37.48053, 126.79422 ...",126.794788,37.480983
1,다사372468,111.0,"MULTIPOLYGON (((126.78934 37.51836, 126.78934 ...",126.789905,37.518810
2,다사351459,63.0,"MULTIPOLYGON (((126.76566 37.51010, 126.76565 ...",126.766222,37.510553
3,다사385461,356.0,"MULTIPOLYGON (((126.80411 37.51213, 126.80410 ...",126.804674,37.512589
4,다사352468,200.0,"MULTIPOLYGON (((126.76671 37.51822, 126.76670 ...",126.767274,37.518672
...,...,...,...,...,...
5620,다사379488,0.0,"MULTIPOLYGON (((126.79710 37.53643, 126.79709 ...",126.797657,37.536883
5621,다사387492,0.0,"MULTIPOLYGON (((126.80612 37.54009, 126.80611 ...",126.806678,37.540542
5622,다사374464,0.0,"MULTIPOLYGON (((126.79164 37.51476, 126.79163 ...",126.792202,37.515219
5623,다사374455,0.0,"MULTIPOLYGON (((126.79172 37.50665, 126.79171 ...",126.792279,37.507107


In [39]:
# 인구분포도에서 val 값이 0인 지역은 설치할 이유가 없기에 제거
nobody = 인구분포도['val']==0.0
nobody
#mock_data[~mask].head() # ~를 포함하게 되면 mask의 값을 제외, ~을 제외하면 mask의 값을 포함입니다.



0       False
1       False
2       False
3       False
4       False
        ...  
5620     True
5621     True
5622     True
5623     True
5624     True
Name: val, Length: 5625, dtype: bool

In [43]:
인구분포도 = 인구분포도[~nobody]

In [44]:
인구분포도

,gid,val,geometry,경도,위도
0,다사376426,721.0,"MULTIPOLYGON (((126.79423 37.48053, 126.79422 ...",126.794788,37.480983
1,다사372468,111.0,"MULTIPOLYGON (((126.78934 37.51836, 126.78934 ...",126.789905,37.518810
2,다사351459,63.0,"MULTIPOLYGON (((126.76566 37.51010, 126.76565 ...",126.766222,37.510553
3,다사385461,356.0,"MULTIPOLYGON (((126.80411 37.51213, 126.80410 ...",126.804674,37.512589
4,다사352468,200.0,"MULTIPOLYGON (((126.76671 37.51822, 126.76670 ...",126.767274,37.518672
...,...,...,...,...,...
2893,다사369466,55.0,"MULTIPOLYGON (((126.78597 37.51653, 126.78596 ...",126.786528,37.516987
2894,다사380460,14.0,"MULTIPOLYGON (((126.79846 37.51120, 126.79846 ...",126.799025,37.511654
2895,다사385434,91.0,"MULTIPOLYGON (((126.80434 37.48780, 126.80433 ...",126.804900,37.488254
2896,다사345436,25.0,"MULTIPOLYGON (((126.75908 37.48933, 126.75907 ...",126.759639,37.489781


In [46]:
인구분포도.describe()

,val,경도,위도
count,2536.000000,2536.000000,2536.000000
mean,323.201893,126.788406,37.497860
std,328.952490,0.021022,0.018829
min,6.000000,126.741505,37.460353
25%,131.000000,126.771947,37.483470
50%,274.000000,126.788675,37.494249
75%,425.000000,126.806764,37.514976
max,5379.000000,126.829961,37.543998


In [57]:
# 상위 25% 지역을 추출함. (425.000000)
인구분포도_25 = 인구분포도.sort_values(by="val", ascending=False)
인구분포도_25 = 인구분포도_25[인구분포도_25["val"] >= 425.0]
인구분포도_25

,gid,val,geometry,경도,위도
2697,다사352458,5379.0,"MULTIPOLYGON (((126.76680 37.50920, 126.76679 ...",126.767363,37.509659
2878,다사337441,4169.0,"MULTIPOLYGON (((126.74998 37.49378, 126.74997 ...",126.750545,37.494231
395,다사349459,3789.0,"MULTIPOLYGON (((126.76340 37.51009, 126.76339 ...",126.763959,37.510539
1327,다사347458,3438.0,"MULTIPOLYGON (((126.76114 37.50917, 126.76114 ...",126.761706,37.509624
1287,다사352450,3211.0,"MULTIPOLYGON (((126.76687 37.50199, 126.76686 ...",126.767433,37.502449
...,...,...,...,...,...
1361,다사368475,425.0,"MULTIPOLYGON (((126.78476 37.52464, 126.78475 ...",126.785319,37.525092
1183,다사358452,425.0,"MULTIPOLYGON (((126.77364 37.50384, 126.77363 ...",126.774203,37.504293
1982,다사351423,425.0,"MULTIPOLYGON (((126.76598 37.47765, 126.76597 ...",126.766540,37.478107
2130,다사343443,425.0,"MULTIPOLYGON (((126.75675 37.49562, 126.75674 ...",126.757314,37.496076


In [58]:
# 상위 5% 지역을 추출함. (425.000000)
인구분포도['val'].quantile(q=0.95, interpolation='nearest')
인구분포도_5 = 인구분포도[인구분포도["val"] >= 768.0]
인구분포도_5

,gid,val,geometry,경도,위도
17,다사353426,871.0,"MULTIPOLYGON (((126.76821 37.48037, 126.76821 ...",126.768775,37.480825
95,다사343428,907.0,"MULTIPOLYGON (((126.75689 37.48210, 126.75688 ...",126.757448,37.482557
112,다사361447,787.0,"MULTIPOLYGON (((126.77708 37.49935, 126.77707 ...",126.777641,37.499808
115,다사383416,865.0,"MULTIPOLYGON (((126.80223 37.47156, 126.80222 ...",126.802788,37.472017
116,다사376442,849.0,"MULTIPOLYGON (((126.79409 37.49495, 126.79408 ...",126.794652,37.495404
...,...,...,...,...,...
2821,다사348432,1009.0,"MULTIPOLYGON (((126.76251 37.48574, 126.76250 ...",126.763067,37.486197
2875,다사351465,1324.0,"MULTIPOLYGON (((126.76561 37.51551, 126.76560 ...",126.766169,37.515961
2878,다사337441,4169.0,"MULTIPOLYGON (((126.74998 37.49378, 126.74997 ...",126.750545,37.494231
2882,다사354451,872.0,"MULTIPOLYGON (((126.76913 37.50291, 126.76912 ...",126.769687,37.503364


In [70]:
인구분포도_5.to_csv("output/인구분포도_5.csv")

In [61]:

# 부천 지도 만들기
import folium
bucheon_map = folium.Map(location=[37.50554861215234,126.77550612183495], zoom_start=13)


for name, lat, lng in zip(인구분포도_5.val, 인구분포도_5.위도, 인구분포도_5.경도):
    folium.CircleMarker([lat, lng],
                        radius=3,         # 원의 반지름
                        fill=True,
                        fill_color='purple',    # 원을 채우는 색
                        fill_opacity=0.7, # 투명도    
                        popup=name
    ).add_to(bucheon_map)

display(bucheon_map)


In [75]:
import pydeck as pdk

layer = pdk.Layer(
    'ScatterplotLayer',  # `type` positional argument is here
    인구분포도_5,
    get_position=['경도', '위도'],
    pickable=True,
    auto_highlight=True,
    get_fill_color=[180, 0, 200, 140],
    get_radius=20)


center = [37.50554861215234,126.77550612183495]

view_state = pdk.ViewState(
    longitude=center[0],
    latitude=center[1],
    min_zoom=5,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36,
    zoom=15)



r = pdk.Deck(layers=[layer], initial_view_state=view_state, map_style='dark')
r.show()

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"initialViewState": {"bea…

In [63]:
def 인구분포도_후보_생성(Data):
    Data_aug = Data.copy()
    ###
    print("-----------기존의 100X100 centroid에서 꼭지점을 이용하여 후보 augmentation------------")
    for x in ['minx', 'maxx']:
        for y in ['miny','maxy']:
            Data_aug_temp = Data.copy()
            exec("Data_aug_temp['경도'] = Data['geometry'].bounds." + x)
            exec("Data_aug_temp['위도'] = Data['geometry'].bounds." + y)
            Data_aug = Data_aug.append(Data_aug_temp)
    ###
    Data_aug = Data_aug.iloc[len(Data):,:]
    Data_aug = Data_aug.reset_index(drop=True)
    
    ###
    near = []
    print("------------새로 만든 후보 인접 4개 인구분포 구하기 -----------------------------------")
    for idx in tqdm(Data_aug.index):
        dist = np.array(Data_aug[['경도','위도']])[idx] - np.array(Data[['경도','위도']])
        dist = (dist**2).sum(axis=1)
        Data_temp = Data.copy()
        Data_temp['dist'] = dist
        near.append(list(np.array(pd.Series(np.array(Data_temp.sort_values('dist')['gid'])[0:4]).sort_values())))
    
    ###
    Data_aug['gid'] = near
    Data_aug = Data_aug.loc[Data_aug['gid'].astype('str').drop_duplicates().index,:]
    Data_aug = Data_aug.reset_index(drop=True)
    
    ###
    
    print("-----------인접 4개 인구분포의 평균으로 후보 augmentation한 val 값 넣어주기 ------------")
    for idx in tqdm(Data_aug.index):
        Data_aug.loc[idx, 'val'] = Data.loc[Data['gid'].isin(Data_aug['gid'][idx]), 'val'].mean()
      
    ###
    
    Data_final = Data.append(Data_aug)
    
    return Data_final

In [69]:
인구분포2 = 인구분포도_후보_생성(인구분포도_5)
인구분포2

-----------기존의 100X100 centroid에서 꼭지점을 이용하여 후보 augmentation------------
------------새로 만든 후보 인접 4개 인구분포 구하기 -----------------------------------



-----------인접 4개 인구분포의 평균으로 후보 augmentation한 val 값 넣어주기 ------------


,gid,val,geometry,경도,위도
17,다사353426,871.00,"MULTIPOLYGON (((126.76821 37.48037, 126.76821 ...",126.768775,37.480825
95,다사343428,907.00,"MULTIPOLYGON (((126.75689 37.48210, 126.75688 ...",126.757448,37.482557
112,다사361447,787.00,"MULTIPOLYGON (((126.77708 37.49935, 126.77707 ...",126.777641,37.499808
115,다사383416,865.00,"MULTIPOLYGON (((126.80223 37.47156, 126.80222 ...",126.802788,37.472017
116,다사376442,849.00,"MULTIPOLYGON (((126.79409 37.49495, 126.79408 ...",126.794652,37.495404
...,...,...,...,...,...
174,"[다사337441, 다사337446, 다사338443, 다사340446]",2029.25,"MULTIPOLYGON (((126.75110 37.49559, 126.75109 ...",126.752228,37.496495
175,"[다사352437, 다사352438, 다사353434, 다사353435]",940.25,"MULTIPOLYGON (((126.76813 37.48848, 126.76813 ...",126.769266,37.489390
176,"[다사341447, 다사343445, 다사345449, 다사346444]",879.25,"MULTIPOLYGON (((126.75673 37.49742, 126.75673 ...",126.757866,37.498333
177,"[다사343445, 다사345449, 다사346455, 다사347448]",1067.50,"MULTIPOLYGON (((126.75896 37.50104, 126.75895 ...",126.760093,37.501952


In [67]:
def make_데이터(경도, 위도):
    batch_size = 10
    
    인구분포도_경위도 = np.stack([인구분포도['geometry'].centroid.x, 인구분포도['geometry'].centroid.y], axis=1).reshape([1, -1, 2])
    전철역_경위도 = np.stack([subway_bucheon['lat'], subway_bucheon['lon']], axis=1)
    버스정류소_경위도 = np.stack([bucheon_bus['WGS84위도'], bucheon_bus['WGS84위도']], axis=1)
    자전거스테이션_경위도 = np.stack([bike_bucheon['위도'], bike_bucheon['위도']], axis=1)  
    데이터 = pd.DataFrame()
    
    for i in tqdm(range(int(np.ceil(len(경도) / batch_size)))):
        #s, e = batch_size * i, batch_size * (i + 1)
        #자전거스테이션_경위도 = np.stack([경도[s:e], 위도[s:e]], axis=1).reshape([-1, 1, 2])
        인구분포_조건 = np.sum((자전거스테이션_경위도 - 인구분포도_경위도)**2, axis=2)**0.5 < 거리 * 축척
        인구분포 = np.array([인구분포도.loc[x, 'val'].mean() for x in 인구분포_조건])
        전철역_조건 = np.any(np.sum((자전거스테이션_경위도 - 전철역_경위도)**2, axis=2)**0.5 < 거리 * 축척, axis=1)
        버스정류소_조건 = np.any(np.sum((자전거스테이션_경위도 - 버스정류소_경위도)**2, axis=2)**0.5 < 거리 * 축척, axis=1)

        
        데이터 = 데이터.append(pd.DataFrame(
            columns=['인구분포', '전철역', '버스정류소'],
            data=np.stack([인구분포2, 전철역_조건, 버스정류소_조건], axis=1),
        ), ignore_index=True)
        
    return 데이터




In [68]:
학습데이터 = make_데이터(bike_bucheon['경도'], bike_bucheon['위도'])
학습데이터.index = 자전거스테이션['Station_ID']
학습데이터

<ipython-input-67-ff685b4c33b4>:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  인구분포도_경위도 = np.stack([인구분포도['geometry'].centroid.x, 인구분포도['geometry'].centroid.y], axis=1).reshape([1, -1, 2])


ValueError: operands could not be broadcast together with shapes (6,2) (1,2536,2) 